In [169]:
from datasets import load_dataset, concatenate_datasets

In [170]:
dataset = load_dataset("slon-hk/BooksSummarizationRU")

In [171]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'Title', 'Author', 'Summary', 'Full Text'],
        num_rows: 1271
    })
    test: Dataset({
        features: ['ID', 'Title', 'Author', 'Summary', 'Full Text'],
        num_rows: 142
    })
})

In [172]:
dataset = dataset.map(lambda x: {"len": len(x["Full Text"])})

In [173]:
full_dataset = concatenate_datasets([dataset["train"], dataset["test"]])

In [174]:
import pandas as pd

df = pd.DataFrame(full_dataset)
df.shape

(1413, 6)

In [175]:
df.isnull().sum()

ID           0
Title        0
Author       0
Summary      0
Full Text    0
len          0
dtype: int64

In [176]:
df['Author'] = df['Author'].str.lower()
df.drop(columns='Summary', inplace=True)

In [177]:
df = (
    df.assign(author_books=lambda d: d['Author'].map(d['Author'].value_counts()))
    .sort_values(by=['Title', 'author_books'], ascending=[True, False])
    .drop_duplicates(subset=['Title', 'Author'], keep='first')
    .drop(columns=['author_books'])
)
df.shape

(649, 5)

In [178]:
df[df.duplicated(subset=['Title'], keep=False)].sort_values('Title')

,ID,Title,Author,Full Text,len
76,738,Кавказский пленник,александр пушкин,Кавказский пленник\nТолстой Лев Николаевич\nКа...,42093
984,1187,Кавказский пленник,саша чёрный,Кавказский пленник\nМихаил Юрьевич Лермонтов\n...,18995
165,1280,Лейли и Меджнун,низами гянджеви,Лейли и Меджнун\nНизами Гянджеви\nЛЕЙЛИ И МЕДЖ...,123367
663,1337,Лейли и Меджнун,мухаммед физули,Лейли и Меджнун\nНизами Гянджеви\nЛЕЙЛИ И МЕДЖ...,123367


In [179]:
import math

CHARS_PER_PAGE = 2000
df['num_pages'] = df['len'].apply(lambda x: math.ceil(x / CHARS_PER_PAGE))
df = df[df['num_pages'] > 1]

In [180]:
from razdel import sentenize

df['num_sentences'] = df['Full Text'].apply(lambda x: len(list(sentenize(x))))

In [181]:
field = 'num_sentences'

median_len = df[field].median()
print(f"Median length: {median_len}")
mean_len = df[field].mean()
print(f"Mean length: {mean_len}")
max_len = df[field].quantile(0.6)
print(f"Max length: {max_len}")
min_len = df[field].quantile(0.1)
print(f"Min length: {min_len}")

Median length: 2246.0
Mean length: 3492.703875968992
Max length: 3159.399999999999
Min length: 155.4


In [182]:
df_selected = df[(df[field] <= max_len) & (df[field] >= min_len)].copy()
print(df_selected.shape)
df_selected[field].describe()

(322, 7)


count     322.000000
mean     1153.937888
std       965.402856
min       156.000000
25%       313.750000
50%       733.000000
75%      1893.500000
max      3147.000000
Name: num_sentences, dtype: float64

In [183]:
df_selected['num_pages'].describe()

count    322.000000
mean      49.357143
std       49.213754
min        5.000000
25%       14.000000
50%       30.000000
75%       68.000000
max      247.000000
Name: num_pages, dtype: float64

In [184]:
df_selected['len'].describe()

count       322.000000
mean      97699.590062
std       98462.552279
min        8041.000000
25%       26218.750000
50%       59423.000000
75%      134330.750000
max      493243.000000
Name: len, dtype: float64

In [185]:
df_selected.reset_index(drop=True, inplace=True)
df_selected[['Title', 'len', 'num_pages', 'num_sentences']].head()

,Title,len,num_pages,num_sentences
0,«Простите нас!»,25903,13,302
1,«Человек за бортом»,36979,19,489
2,Авария,12090,7,179
3,Адский житель,59232,30,435
4,Александрийский шейх Али-Бану и его невольники,149352,75,1448


In [186]:
df_selected

,ID,Title,Author,Full Text,len,num_pages,num_sentences
0,205,«Простите нас!»,юрий бондарев,«Простите нас!»\nЮрий Бондарев\n«ПРОСТИТЕ НАС!...,25903,13,302
1,1373,«Человек за бортом»,к. м. станюкович,"'Человек за бортом!'\nСпасибо, что скачали кни...",36979,19,489
2,236,Авария,фридрих дюрренматт,Авария\nЗегальский Витольд\nАвария\nВИТОЛЬД ЗЕ...,12090,7,179
3,879,Адский житель,ф. де ла м. фуке,Адский житель\nДе Ла Мотт Фуке Фридрих\nАдский...,59232,30,435
4,1334,Александрийский шейх Али-Бану и его невольники,вильгельм гауф,Александрийский шейх Али-Бану и его невольники...,149352,75,1448
...,...,...,...,...,...,...,...
317,854,Эдип в Колоне,софокл,Эдип в Колоне\nСофокл\nЭдип в Колоне\nТрагедия...,176829,89,3067
318,476,Эжени де Франваль,маркиз де сад,Эжени де Франваль\nМаркиз де Сад\nЭжени де Фра...,140876,71,1309
319,1365,Этрусская ваза,проспер мериме,Этрусская ваза\nПроспер Мериме\nЭТРУССКАЯ ВАЗА...,48527,25,642
320,55,Я в замке король,сьюзен хилл,Кто в замке король\nСергей СМИРНОВ\n(под псевд...,155777,78,2766


In [187]:
df_selected.to_json('selected.jsonl', orient="records", lines=True, force_ascii=False)

# CLEANED

In [1]:
import pandas as pd
df_loaded = pd.read_json("selected.jsonl", lines=True, encoding="utf-8")

In [2]:
df_loaded

,ID,Title,Author,Full Text,len,num_pages,num_sentences
0,205,«Простите нас!»,юрий бондарев,Южный экспресс задержался здесь не более пяти ...,25903,13,302
1,1373,«Человек за бортом»,к. м. станюкович,Жapa тропического дня начинала спадать. Солнце...,36979,19,489
2,236,Авария,фридрих дюрренматт,Он проснулся от звука зуммера. Тупо стал водит...,12090,7,179
3,879,Адский житель,ф. де ла м. фуке,Однажды под вечер приехал в Венецию - всем изв...,59232,30,435
4,1334,Александрийский шейх Али-Бану и его невольники,вильгельм гауф,Cтранным человеком был александрийский шейх Ал...,149352,75,1448
...,...,...,...,...,...,...,...
308,693,Чёрная стрела,р. л. стивенсон,Люблю звёзды. Могу смотреть на них бесконечно....,18949,10,428
309,121,Шесть персонажей в поисках автора,луиджи пиранделло,Персонажи еще не написанной комедии\nОтец.\nМа...,153258,77,2968
310,476,Эжени де Франваль,маркиз де сад,Движимые исключительно стремлением наставлять ...,140876,71,1309
311,1365,Этрусская ваза,проспер мериме,Огюста Сен-Клера не любили в так называемом «б...,48527,25,642


In [3]:
df_loaded['len'] = df_loaded['Full Text'].str.len()
df_loaded['len'].sum()

30066892

In [4]:
df_loaded['Full Text'] = df_loaded['Full Text'].str.replace(r'\[.*?\]', '', regex=True)

In [5]:
df_loaded['len'] = df_loaded['Full Text'].str.len()
df_loaded['len'].sum()

30048004

In [6]:
df_loaded.columns

Index(['ID', 'Title', 'Author', 'Full Text', 'len', 'num_pages',
       'num_sentences'],
      dtype='object')

In [7]:
import time
import requests
import os
import json

UDPIPE_PORT = 8001
def process_text_local_udpipe(text, port=UDPIPE_PORT, output_format='conllu'):
    """
    Обрабатывает текст с помощью локального сервера UDPipe.
    """
    url = f'http://localhost:{port}/process'
    params = {
        'tokenizer': 'ranges',
        'tagger': '',
        'parser': '',
        'data': text,
        'output': output_format,
    }
    try:
        response = requests.post(url, data=params)
        response.raise_for_status()
        result_json = response.json()
        if 'result' in result_json:
            return result_json['result']
        else:
            raise Exception(f"Ключ 'result' не найден в ответе от сервера: {result_json}")
    except json.JSONDecodeError:
        raise Exception(f"Сервер вернул невалидный JSON. Текст ответа: {response.text}")
    except requests.exceptions.RequestException as e:
        raise Exception(f"Ошибка при подключении к локальному серверу UDPipe: {e}")

if not os.path.exists('books'):
    os.mkdir('books')
for index, row in df_loaded.iterrows():
    print(index, row['Title'], row['len'], row['num_sentences'])
    text = row['Full Text']
    conllu_output = process_text_local_udpipe(text)
    id = row['ID']
    with open(f'books/{id}.conllu', 'w') as f:
        f.write(conllu_output)
    time.sleep(5)

0 «Простите нас!» 25857 302
1 «Человек за бортом» 36697 489
2 Авария 12028 179
3 Адский житель 59140 435
4 Александрийский шейх Али-Бану и его невольники 149129 1448
5 Аленький цветочек 43399 252
6 Альберт 50593 650
7 Алюминиевое солнце 62589 903
8 Алёшкино сердце 24386 368
9 Американские партизаны 230563 3003
10 Анна Снегина 19158 477
11 Антон-Горемыка 205551 1904
12 Арбузная корка 18689 175
13 Баргамот и Гараська 21135 214
14 Барышня-крестьянка 35585 410
15 Бататовая каша 34429 387
16 Бахчисарайский фонтан 19101 248
17 Бедность не порок 76983 2359
18 Безлюбый 65825 793
19 Белая цапля 21798 275
20 Белый пудель 60615 923
21 Бобыль 38230 331
22 Британик 18105 227
23 Бузинная матушка 14096 200
24 Буриданов осёл 473705 3011
25 В прекрасном и яростном мире
Машинист Мальцев 25287 290
26 В чаще 17496 287
27 Ваганов 16459 239
28 Ванина Ванини 52679 630
29 Василий Тёркин 127955 2288
30 Веер леди Уиндермир 104698 2995
31 Великая шахматная доска
Господство Америки и его геостратегические императ